# **DSR 36 - Mini Competition (Oct 5-7th, 2023): Richter's Earthquake Damage Predictor**

![Eearthquake](images/earthquake.jpg)


## Team Remote: Boris, David, Petr, Milosz
- Working with agile principles
- Using Google Meets and Jira
- Dividing roles Admin: Milosz -- EDA/Viz: Petr -- Modeling: Boris -- Feature Engineering: David


## Git Repository

https://github.com/miloszpaul/minicomp


![Our Repo](images/repo.png)


# In this competition, we aim to predict the level of damage to buildings caused by the 2015 Gorkha earthquake in Nepal

![Map](images/map.png)

# Our given dataset includes 38 features related to 162k buildings. 

# Our main hypothesis is that destruction is based on building **location** and **construction**. 

## Location (geo_level_1_id)

![Map](images/nepal.png)
Source: Wikimedia 

## Construction types (foundation_type, roof_type, ground_floor_type, etc)


![Map](images/building_types.jpg)
Source: https://doi.org/10.1016/j.engstruct.2016.04.043


# The **target variable** is _'damage_grade'_, which represents the level of damage to the building. There are 3 grades: 

# 1 (low damage)
# 2 (medium damage)
# 3 (almost complete destruction)

# Our model is optimized lightGBM and received a test score of ....

![Submission](images/submission.png)

# In this presentation we walk you through the 6 steps of our model. 

# 0. Import modules and data

In [176]:
# Modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [177]:
# Import scripts
import helper_functions # Various helper functions
import log_regression # Simple regression model
import lgb_optimized # Random forest

In [178]:
# Load data
X, y, X_test = helper_functions.imports()

## 1. Perform exploratory **data analysis** to understand the dataset's characteristics, distributions, and relationships between variables.



In [179]:
print(f"Proportions of the DataFrame X, containing the features for testing: {X.shape}")
print(f"Proportions of the DataFrage y, containing the target value for testing: {y.shape}")
print(f"Proportions of the DataFrame X, containing the features for the prediction: {X_test.shape}")

Proportions of the DataFrame X, containing the features for testing: (260601, 39)
Proportions of the DataFrage y, containing the target value for testing: (260601, 2)
Proportions of the DataFrame X, containing the features for the prediction: (86868, 39)


In [180]:
X.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [181]:
y.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [182]:
X_test.head()


,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


## Check whether _X_ and _X_test_ have the same columns

In [183]:
X.columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_i

In [184]:
helper_functions.test_column_equality(X, X_test)

AttributeError: module 'helper_functions' has no attribute 'test_column_equality'

In [ ]:
X.dtypes

building_id                                int64
geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_c

In [ ]:
X.nunique()

has_superstructure_adobe_mud               2
age                                       42
count_floors_pre_eq                        9
area_percentage                           84
height_percentage                         27
has_secondary_use                          2
has_superstructure_cement_mortar_brick     2
has_superstructure_timber                  2
has_superstructure_bamboo                  2
dtype: int64

Note: From the description, we assume Geo Level 3 is the most precise whereas Geo Level 1 the least precise. Follow that logic, we expect more unique data points in Level 3 than Level 1.

In [ ]:
print(f"Unique data points in geo_level_1_id: {X.loc[:, 'geo_level_1_id'].nunique()}")
print(f"Unique data points in geo_level_2_id: {X.loc[:, 'geo_level_2_id'].nunique()}")
print(f"Unique data points in geo_level_3_id: {X.loc[:, 'geo_level_3_id'].nunique()}")

Unique data points in geo_level_1_id: 31
Unique data points in geo_level_2_id: 1414
Unique data points in geo_level_3_id: 11595


!!!MISSING (Report) on MISSING DATA? 

## 2. **Clean the data** to prepare it for modeling.

In [ ]:
#First cleaning (before split - if required)
#For this competition, we initially focus on predicting 'damage_grade.' So, we extract this target variable from the dataset.

y = y['damage_grade']

In [ ]:
# Split data in training and validation set to evaluate our model's performance.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## _geo_level2_ mean encodings

In [ ]:
geo_levels = []
for col in filter(lambda col: col.startswith("geo"), X):
    print(str(col))
    geo_levels.append(col)

In [ ]:
mean_encodings = data.groupby('geo_level_2_id')['damage_grade'].mean()
pd.DataFrame(mean_encodings).head()


NameError: name 'data' is not defined

In [ ]:
for df in X, X_test:
    df["geo_level_2_enc"] = df['geo_level_2_id'].map(mean_encodings)
    # print(df[["geo_level_2_id", "geo_level_2_enc"]].sort_values("geo_level_2_id")[:100], end="\n")

print(geo_levels)

KeyError: 'geo_level_2_id'

## _geo_level1_ dummies

In [ ]:
dummies = pd.get_dummies(X["geo_level_1_id"], prefix="geo_level_cat")
X = pd.concat([X, dummies], axis=1)
dummies = pd.get_dummies(X_test["geo_level_1_id"], prefix="geo_level_cat")
X_test = pd.concat([X_test, dummies], axis=1)

print(
    X.shape,    
    X_test.shape, sep="\n"
)

KeyError: 'geo_level_1_id'

In [ ]:
geo_columns = [col for col in X.columns if col.startswith('geo_level_cat')]
print(X.groupby("geo_level_1_id").first()[geo_columns].iloc[:6,:6])

KeyError: 'geo_level_1_id'

## _foundation_type_ dummies

In [ ]:
X = pd.get_dummies(X, columns=["foundation_type"], drop_first=False)
X_test = pd.get_dummies(X_test, columns=["foundation_type"], drop_first=False)

print(X.shape)
print(X_test.shape)
foundation_cols = [col for col in X.columns if col.startswith("foundation")]
print(
    X[foundation_cols].columns,
    X_test[foundation_cols].columns,
    sep="\n")

KeyError: "None of [Index(['foundation_type'], dtype='object')] are in the [columns]"

In [ ]:
X = pd.get_dummies(X, columns=["roof_type"], drop_first=False)
X_test = pd.get_dummies(X_test, columns=["roof_type"], drop_first=False)

print(X.shape)
print(X_test.shape)
cols = [col for col in X.columns if col.startswith("roof_type")]
print(
    X[cols].columns,
    X_test[cols].columns,
    sep="\n")

KeyError: "None of [Index(['roof_type'], dtype='object')] are in the [columns]"

In [ ]:
X = pd.get_dummies(X, columns=["ground_floor_type"], drop_first=False)
X_test = pd.get_dummies(X_test, columns=["ground_floor_type"], drop_first=False)

print(X.shape)
print(X_test.shape)
cols = [col for col in X.columns if col.startswith("ground_floor_type")]
print(
    X[cols].columns,
    X_test[cols].columns,
    sep="\n")

KeyError: "None of [Index(['ground_floor_type'], dtype='object')] are in the [columns]"

# Hot encodings

In [ ]:
for name,df in {"X":X, "X_test":X_test}.items():
    encode_cols = ["foundation_type", "roof_type", "ground_floor_type", "geo_level_1_id"]
    globals()[name] = pd.get_dummies(df, columns=encode_cols, drop_first=False)
    globals()[name].drop(columns=['geo_level_2_id', 'geo_level_3_id'], inplace=True)
    cols = [col for col in df.columns if any(map(col.startswith, encode_cols))]

In [ ]:
for df in X, X_test:
    print(
        df.shape,
        # df.columns,
        df.info()
    )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 9 columns):
 #   Column                                  Non-Null Count   Dtype
---  ------                                  --------------   -----
 0   has_superstructure_adobe_mud            260601 non-null  int64
 1   age                                     260601 non-null  int64
 2   count_floors_pre_eq                     260601 non-null  int64
 3   area_percentage                         260601 non-null  int64
 4   height_percentage                       260601 non-null  int64
 5   has_secondary_use                       260601 non-null  int64
 6   has_superstructure_cement_mortar_brick  260601 non-null  int64
 7   has_superstructure_timber               260601 non-null  int64
 8   has_superstructure_bamboo               260601 non-null  int64
dtypes: int64(9)
memory usage: 17.9 MB
(260601, 9) None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86868 entries, 0 to 86867
Data colum

## Prepare data for analysis

In [ ]:
# set index to building_id
for df in y, X, X_test:
    df.set_index("building_id", inplace=True)

drop_cols = ["land_surface_condition", "other_floor_type", "position", 
             "plan_configuration", "legal_ownership_status"]
for df in X, X_test:
    df.drop(columns=drop_cols, inplace=True)

KeyError: "None of ['building_id'] are in the columns"

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_pred = X_test

# for k,v in {"X":X, X_train, X_valid, X_test}:
for name,df in {"X":X, "X_train":X_train, "X_valid": X_valid, "X_test": X_test}.items():
    print(name, *df.shape, sep="\t")

X	260601	9
X_train	208480	9
X_valid	52121	9
X_test	86868	77


## 3. **Visualizations**

![Feature Correlations](images/feature_corr.png)

## **Plot:** Correlation heatmap of predictors

## **Observations:** Secondary uses are highly correlated, Ground floor and height are highly correlated

## **Actions:** 1) eliminate one variable (gorund floor OR height), or 2) keep both but be aware that the coefficients are not statistically significant, however combined predictive power might be worthwhile to keep

![Historgrams](images/histrograms.png)

## **Plot I:** Histograms of geo_id

## **Observations:** Epicenter of earthquake is at 17

## **Actions:** Confirmation of hypothesis that location is important

## **Plot II:** Histograms of age 

## **Observations:** 1) Correlation between newest buildings and low damage 1, i.e. the percentage of low damage decreases with age; 1.1) very low number of buildings explains varation around age 200 2) outliers at age 1000 are old protected buildings 


## **Actions:** Non linear model, use _lightgbm_ 


![Pair Correlation](images/pair_corr.png)

## **Plot:** Pairwise correlation of each feature

## **Observations:** Outliers (for instance superstructures), are highly correlated to the predictor

## **Actions:** Not blindly exclude outliers  

![Feature Importance](images/feature_imp.png)

## **Plot:** Feature importance of our most relevant model lightgbm classifier 

## **Observations:** How frequent is it that we ask to split forest by age? 

## **Actions:** 

## 4. Generate a **Random Forest model** and optimize it for better predictive performance.



In [196]:
# Initially, we limit the dataset to a subset of relevant features for testing tree generation. 
# These features are selected based on our domain knowledge and preliminary analysis.


mask = ['has_superstructure_adobe_mud', 'age','count_floors_pre_eq','area_percentage','height_percentage','has_secondary_use',
        'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo'] 


# Apply the feature mask
X = X[mask]

X_train = X_train[mask]
X_valid = X_valid[mask]

# Prepare the test data with the same feature mask
X_test_pred = X_test[mask]

In [197]:
# Initialize a LightGBM (LGBM) model using the training and validation sets.

model = lgb_optimized.LGBM(X_train, X_valid, y_train, y_valid)


# Generate and optimize the Random Forest model and feed it with test data to create predictions.

y_pred = model.optimization(X, y, X_test_pred)

[I 2023-10-07 14:03:46,193] A new study created in memory with name: no-name-12414550-cc1f-48e2-9d3a-f498eecdd3d5


[1]	valid_0's multi_logloss: 0.878659
[2]	valid_0's multi_logloss: 0.859845
[3]	valid_0's multi_logloss: 0.84838
[4]	valid_0's multi_logloss: 0.840845
[5]	valid_0's multi_logloss: 0.835711
[6]	valid_0's multi_logloss: 0.832167
[7]	valid_0's multi_logloss: 0.829411
[8]	valid_0's multi_logloss: 0.827403
[9]	valid_0's multi_logloss: 0.825927
[10]	valid_0's multi_logloss: 0.824816
[11]	valid_0's multi_logloss: 0.823814
[12]	valid_0's multi_logloss: 0.823098
[13]	valid_0's multi_logloss: 0.822551
[14]	valid_0's multi_logloss: 0.822134
[15]	valid_0's multi_logloss: 0.821843
[16]	valid_0's multi_logloss: 0.821499
[17]	valid_0's multi_logloss: 0.821311
[18]	valid_0's multi_logloss: 0.821073
[19]	valid_0's multi_logloss: 0.820993
[20]	valid_0's multi_logloss: 0.820925
[21]	valid_0's multi_logloss: 0.820865
[22]	valid_0's multi_logloss: 0.820809
[23]	valid_0's multi_logloss: 0.82077
[24]	valid_0's multi_logloss: 0.820691
[25]	valid_0's multi_logloss: 0.820717
[26]	valid_0's multi_logloss: 0.8206

[I 2023-10-07 14:03:49,430] Trial 0 finished with value: 0.293279100554479 and parameters: {'learning_rate': 0.25377560330393273, 'subsample': 0.9763214085156694, 'num_leaves': 69, 'min_data_in_leaf': 3, 'max_depth': 8, 'lambda_l2': 0.36708550397144457}. Best is trial 0 with value: 0.293279100554479.


[1]	valid_0's multi_logloss: 0.896091
[2]	valid_0's multi_logloss: 0.881734
[3]	valid_0's multi_logloss: 0.870677
[4]	valid_0's multi_logloss: 0.862398
[5]	valid_0's multi_logloss: 0.855999
[6]	valid_0's multi_logloss: 0.850985
[7]	valid_0's multi_logloss: 0.846863
[8]	valid_0's multi_logloss: 0.843596
[9]	valid_0's multi_logloss: 0.840659
[10]	valid_0's multi_logloss: 0.838297
[11]	valid_0's multi_logloss: 0.836339
[12]	valid_0's multi_logloss: 0.834694
[13]	valid_0's multi_logloss: 0.833305
[14]	valid_0's multi_logloss: 0.832016
[15]	valid_0's multi_logloss: 0.831012
[16]	valid_0's multi_logloss: 0.829889
[17]	valid_0's multi_logloss: 0.829045
[18]	valid_0's multi_logloss: 0.828399
[19]	valid_0's multi_logloss: 0.827794
[20]	valid_0's multi_logloss: 0.827216
[21]	valid_0's multi_logloss: 0.826624
[22]	valid_0's multi_logloss: 0.826137
[23]	valid_0's multi_logloss: 0.825753
[24]	valid_0's multi_logloss: 0.825394
[25]	valid_0's multi_logloss: 0.825148
[26]	valid_0's multi_logloss: 0.82

[I 2023-10-07 14:03:51,470] Trial 1 finished with value: 0.30260355710749987 and parameters: {'learning_rate': 0.14907288454714238, 'subsample': 0.8875697168641824, 'num_leaves': 22, 'min_data_in_leaf': 1, 'max_depth': 14, 'lambda_l2': 0.8456162132831313}. Best is trial 1 with value: 0.30260355710749987.
/Users/miloszrosi/anaconda3/envs/dsr-setup/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/miloszrosi/anaconda3/envs/dsr-setup/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Number of finished trials: 2
Best trial:
--------------------------------
Best F1 Score: 0.30260355710749987
--------------------------------
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] lambda_l2 is set=0.8456162132831313, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8456162132831313


AttributeError: 'LGBM' object has no attribute 'optimization'

## 5. **Evaluation** (on f1 score)

## 6. **Export**

In [ ]:
# Export the model's predictions to create a CSV file following DrivenData.org data standards.

helper_functions.write_output(X_test, y_pred)